# V4V Experiment Evaluation

In [ ]:
import numpy as np
import pandas as pd
import respiration.utils as utils

signals_dir = utils.dir_path('outputs', 'signals')

In [ ]:
rhythm_former_path = utils.join_paths(signals_dir, 'rhythm_former_v4v.csv')

rhythm_former = pd.read_csv(rhythm_former_path)
rhythm_former['signal'] = rhythm_former['signal'].apply(eval).apply(np.array)

# Only keep the columns that are needed
rhythm_former.head()

In [ ]:
from respiration.analysis import frequency_from_psd

sampling_rate = 30
lowpass = 0.1
highpass = 0.5

rhythm_former['predicted_rr'] = rhythm_former['signal'].apply(
    lambda x: frequency_from_psd(x, sampling_rate, lowpass, highpass) * 60,
)

In [ ]:
rhythm_former.head()

In [ ]:
from respiration.dataset import V4VDataset

dataset = V4VDataset()
metadata = pd.DataFrame(dataset.get_metadata())

# Remove all HR rows
metadata = metadata[metadata['vital'] != 'HR']

# Rename the signal to gt_signal
metadata = metadata.rename(columns={'signal': 'gt_signal'})

# Add the average respiration rate to the metadata
metadata['gt_rr'] = metadata['gt_signal'].apply(lambda x: x.mean())

metadata.head()

In [ ]:
# Merge the two dataframes
merged = pd.merge(rhythm_former, metadata, on='video')
merged.head()

In [ ]:
# Calculate the MAE for each model
from sklearn.metrics import mean_absolute_error

for model in merged['model'].unique():
    model_data = merged[merged['model'] == model]
    mae = mean_absolute_error(model_data['predicted_rr'], model_data['gt_rr'])
    print(f'{model}: {mae}')